# **Penting**
- Jangan mengubah atau menambahkan cell text yang sudah disediakan, Anda hanya perlu mengerjakan cell code yang sudah disediakan.
- Pastikan seluruh kriteria memiliki output yang sesuai, karena jika tidak ada output dianggap tidak selesai.
- Misal, Anda menggunakan df = df.dropna() silakan gunakan df.isnull().sum() sebagai tanda sudah berhasil. Silakan sesuaikan seluruh output dengan perintah yang sudah disediakan.
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Pastikan Anda menggunakan variabel df dari awal sampai akhir dan tidak diperbolehkan mengganti nama variabel tersebut.
- Hapus simbol pagar (#) pada kode yang bertipe komentar jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan
- Pastikan Anda mengerjakan sesuai section yang sudah diberikan tanpa mengubah judul atau header yang disediakan.

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [236]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import joblib

# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [237]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
# Silakan gunakan dataset data_clustering jika tidak menerapkan Interpretasi Hasil Clustering [Advanced]
# Silakan gunakan dataset data_clustering_inverse jika menerapkan Interpretasi Hasil Clustering [Advanced]
# Lengkapi kode berikut
df = pd.read_csv('data_clustering.csv') 

In [238]:
# Tampilkan 5 baris pertama dengan function head.
df.head()

,TransactionAmount,TransactionDate,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,AgeGroup,AmountGroup,Target
0,0.007207,680,1,36,0,0.838710,0,0.244828,0.0,0.336832,105,0,0,0
1,0.195940,1178,1,15,0,0.806452,0,0.451724,0.0,0.918055,192,0,0,0
2,0.065680,1262,1,23,2,0.016129,3,0.158621,0.0,0.068637,41,0,0,0
3,0.096016,818,1,33,2,0.129032,3,0.051724,0.0,0.569198,163,0,0,0
4,0.047888,635,1,28,0,0.000000,3,0.558621,0.0,0.045738,13,1,0,0


In [239]:
# Mengidentifikasi dan menghapus kolom non-numerik (selain 'Target')
# Model machine learning hanya bisa memproses data numerik
cols_to_drop = df.select_dtypes(include=['object', 'datetime64']).columns
# Pastikan kita tidak menghapus kolom target jika tipenya object
cols_to_drop = cols_to_drop.drop('Target') if 'Target' in cols_to_drop else cols_to_drop

if not cols_to_drop.empty:
    df = df.drop(columns=cols_to_drop)
    print(f"Kolom non-numerik yang dihapus: {list(cols_to_drop)}")
else:
    print("Tidak ada kolom non-numerik yang perlu dihapus.")

print("\nData setelah preprocessing (5 baris pertama):")
display(df.head())
print("\nInformasi DataFrame setelah preprocessing:")
df.info()

Tidak ada kolom non-numerik yang perlu dihapus.

Data setelah preprocessing (5 baris pertama):


,TransactionAmount,TransactionDate,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,AgeGroup,AmountGroup,Target
0,0.007207,680,1,36,0,0.838710,0,0.244828,0.0,0.336832,105,0,0,0
1,0.195940,1178,1,15,0,0.806452,0,0.451724,0.0,0.918055,192,0,0,0
2,0.065680,1262,1,23,2,0.016129,3,0.158621,0.0,0.068637,41,0,0,0
3,0.096016,818,1,33,2,0.129032,3,0.051724,0.0,0.569198,163,0,0,0
4,0.047888,635,1,28,0,0.000000,3,0.558621,0.0,0.045738,13,1,0,0



Informasi DataFrame setelah preprocessing:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2398 entries, 0 to 2397
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TransactionAmount        2398 non-null   float64
 1   TransactionDate          2398 non-null   int64  
 2   TransactionType          2398 non-null   int64  
 3   Location                 2398 non-null   int64  
 4   Channel                  2398 non-null   int64  
 5   CustomerAge              2398 non-null   float64
 6   CustomerOccupation       2398 non-null   int64  
 7   TransactionDuration      2398 non-null   float64
 8   LoginAttempts            2398 non-null   float64
 9   AccountBalance           2398 non-null   float64
 10  PreviousTransactionDate  2398 non-null   int64  
 11  AgeGroup                 2398 non-null   int64  
 12  AmountGroup              2398 non-null   int64  
 13  Target                   2398 non-

# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [240]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.
X = df.drop('Target', axis=1)
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nJumlah data latih dan data uji:")
print(f"Jumlah data latih (X_train): {X_train.shape[0]}")
print(f"Jumlah data uji (X_test): {X_test.shape[0]}")


Jumlah data latih dan data uji:
Jumlah data latih (X_train): 1918
Jumlah data uji (X_test): 480


# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [241]:
# Buatlah model klasifikasi menggunakan Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

y_pred_dt = dt_model.predict(X_test)

print("\n--- Evaluasi Model Decision Tree ---")
print(f"Akurasi: {accuracy_score(y_test, y_pred_dt):.4f}")
print(f"Presisi: {precision_score(y_test, y_pred_dt, average='weighted', zero_division=0):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_dt, average='weighted', zero_division=0):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_dt, average='weighted', zero_division=0):.4f}")


--- Evaluasi Model Decision Tree ---
Akurasi: 0.9979
Presisi: 0.9979
Recall: 0.9979
F1-Score: 0.9979


In [242]:
# Menyimpan Model
joblib.dump(dt_model, 'decision_tree_model.h5')
print("\nModel Decision Tree berhasil disimpan sebagai 'decision_tree_model.h5'.")


Model Decision Tree berhasil disimpan sebagai 'decision_tree_model.h5'.


# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [243]:
# Melatih model menggunakan algoritma klasifikasi scikit-learn selain Decision Tree.
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)


In [244]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.
print("\n--- Evaluasi Model Random Forest ---")
print(f"Akurasi: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Presisi: {precision_score(y_test, y_pred_rf, average='weighted', zero_division=0):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf, average='weighted', zero_division=0):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_rf, average='weighted', zero_division=0):.4f}")


--- Evaluasi Model Random Forest ---
Akurasi: 1.0000
Presisi: 1.0000
Recall: 1.0000
F1-Score: 1.0000


In [245]:
# Menyimpan Model Selain Decision Tree
joblib.dump(rf_model, 'explore_RandomForest_classification.h5')
print("\nModel Random Forest berhasil disimpan sebagai 'explore_RandomForest_classification.h5'.")


Model Random Forest berhasil disimpan sebagai 'explore_RandomForest_classification.h5'.


Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [246]:
# Lakukan Hyperparameter Tuning dan Latih ulang.
# Lakukan dalam satu cell ini saja.
print("\n--- Melakukan Hyperparameter Tuning pada Random Forest ---")
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5]
}
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_rf_model = grid_search.best_estimator_
print("\nHyperparameter terbaik:", grid_search.best_params_)


--- Melakukan Hyperparameter Tuning pada Random Forest ---
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=10, min_samples_split=5, n_estimators=100; total time=   0.3s
[CV] END max_depth=10, min_samples_split=2, n_estimators=200; total time=   0.5s
[CV] END max_depth=10, min_samples_split=5, n_estimators=100; total time=   0.3s
[CV] END max_depth=10, min_samples_split=2, n_estimators=200; total time=   0.4s
[CV] END max_depth=10, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END max_depth=10, min_samples_split=2, n_estimators=200; total time=   0.5s
[CV] END max_depth=20, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=10, min_samples_split=5, n_estimators=200; total t

In [247]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada algoritma yang sudah dituning.
y_pred_tuned = best_rf_model.predict(X_test)
print("\n--- Evaluasi Model Random Forest (Setelah Tuning) ---")
print(f"Akurasi: {accuracy_score(y_test, y_pred_tuned):.4f}")
print(f"Presisi: {precision_score(y_test, y_pred_tuned, average='weighted', zero_division=0):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_tuned, average='weighted', zero_division=0):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_tuned, average='weighted', zero_division=0):.4f}")


--- Evaluasi Model Random Forest (Setelah Tuning) ---
Akurasi: 0.9979
Presisi: 0.9979
Recall: 0.9979
F1-Score: 0.9979


In [248]:
# Menyimpan Model hasil tuning
joblib.dump(best_rf_model, 'tuning_classification.h5')
print("\nModel hasil tuning berhasil disimpan sebagai 'tuning_classification.h5'.")


Model hasil tuning berhasil disimpan sebagai 'tuning_classification.h5'.


End of Code